<a href="https://colab.research.google.com/github/PriyathamVarma/PriyathamVarma/blob/main/chat_bot_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is for creating a NLP chatbot.

In [397]:
# Creating the json file bot responses

In [398]:
import random
import numpy as np
import json
import pickle

import nltk
from nltk.stem import WordNetLemmatizer


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


In [399]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [400]:
lemmatizer = WordNetLemmatizer()

In [401]:
intents = json.loads(open('/content/sample_data/sample.json').read())

In [402]:
intents

{'inents': [{'patterns': ['hai', 'hello', 'hey'],
   'responses': ['hello', 'whats up'],
   'tag': 'greetings'},
  {'patterns': ['oy', 'hurray', 'voila'],
   'responses': ['oyyyy', 'mate'],
   'tag': 'answers'}]}

In [403]:
words = []
classes = []
documents = []
ignore_letters = ['?',',','.','!']

In [404]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [405]:
from nltk.tokenize import word_tokenize

In [406]:
for intent in intents['inents']:
  print(intent)
  for patterns in intent['patterns']:
    print(patterns)
    word_list = nltk.word_tokenize(patterns)
    words.extend(word_list)
    print(words)
    #documents
    documents.append((word_list,intent['tag']))
    #classes
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


{'tag': 'greetings', 'patterns': ['hai', 'hello', 'hey'], 'responses': ['hello', 'whats up']}
hai
['hai']
hello
['hai', 'hello']
hey
['hai', 'hello', 'hey']
{'tag': 'answers', 'patterns': ['oy', 'hurray', 'voila'], 'responses': ['oyyyy', 'mate']}
oy
['hai', 'hello', 'hey', 'oy']
hurray
['hai', 'hello', 'hey', 'oy', 'hurray']
voila
['hai', 'hello', 'hey', 'oy', 'hurray', 'voila']


In [407]:
words

['hai', 'hello', 'hey', 'oy', 'hurray', 'voila']

In [408]:
documents

[(['hai'], 'greetings'),
 (['hello'], 'greetings'),
 (['hey'], 'greetings'),
 (['oy'], 'answers'),
 (['hurray'], 'answers'),
 (['voila'], 'answers')]

In [409]:
classes

['greetings', 'answers']

#Lemmatization

In [410]:
words_new = []
for words in words:
  words_new.append(words)
  print(words)
  

hai
hello
hey
oy
hurray
voila


In [411]:
words_new

['hai', 'hello', 'hey', 'oy', 'hurray', 'voila']

In [412]:
lemmatized_words = [lemmatizer.lemmatize(word) for word in words_new if word not in ignore_letters]

In [413]:
sorted(set(lemmatized_words))

['hai', 'hello', 'hey', 'hurray', 'oy', 'voila']

In [414]:
classes = sorted(set(classes))

In [415]:
classes

['answers', 'greetings']

In [416]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# Machine learning part

In [417]:
train = []
output_empty = [0] *len(classes)

In [418]:
output_empty

[0, 0]

In [419]:
documents

[(['hai'], 'greetings'),
 (['hello'], 'greetings'),
 (['hey'], 'greetings'),
 (['oy'], 'answers'),
 (['hurray'], 'answers'),
 (['voila'], 'answers')]

In [420]:
for documents in documents:
  bag = []
  word_patterns = documents[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  #output
  output_row = list(output_empty)
  output_row[classes.index(documents[1])] = 1
  train.append([bag,output_row])


In [421]:
random.shuffle(train)
training = np.array(train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [422]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [423]:
train_y

[[0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1]]

# Model development

In [424]:
model = Sequential()
model.add(Dense(128,input_shape = (len(train_x[0]),), activation = 'relu' ))

In [425]:
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

In [426]:
model.add(Dense(len(train_y[0]), activation='softmax'))

In [427]:
sgd = SGD(lr = 0.01, decay = 1e-6 , momentum=0.9,nesterov=True)
model.compile(loss="categorical_crossentropy",optimizer=sgd, metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [428]:
"""hist = model.fit(np.array(train_x),np.array(train_y),epochs=100,batch_size = 5, verbose=1)
model.save('DS_Bot.model')"""

Epoch 1/100
2/2 [==============================] - 1s 8ms/step - loss: 0.6935 - accuracy: 0.3333
Epoch 2/100
2/2 [==============================] - 0s 7ms/step - loss: 0.6940 - accuracy: 0.5000
Epoch 3/100
2/2 [==============================] - 0s 7ms/step - loss: 0.6934 - accuracy: 0.5000
Epoch 4/100
2/2 [==============================] - 0s 5ms/step - loss: 0.6928 - accuracy: 0.5000
Epoch 5/100
2/2 [==============================] - 0s 7ms/step - loss: 0.6926 - accuracy: 0.5000
Epoch 6/100
2/2 [==============================] - 0s 8ms/step - loss: 0.6955 - accuracy: 0.5000
Epoch 7/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6939 - accuracy: 0.5000
Epoch 8/100
2/2 [==============================] - 0s 7ms/step - loss: 0.6942 - accuracy: 0.5000
Epoch 9/100
2/2 [==============================] - 0s 6ms/step - loss: 0.6949 - accuracy: 0.5000
Epoch 10/100
2/2 [==============================] - 0s 8ms/step - loss: 0.6990 - accuracy: 0.5000
Epoch 11/100
2/2 [===========

In [437]:
hist = model.fit(np.array(train_x),np.array(train_y),epochs=100,batch_size = 5, verbose=1)
model.save('DS_Bot.h5',hist)

Epoch 1/100
2/2 [==============================] - 0s 8ms/step - loss: 0.6991 - accuracy: 0.5000
Epoch 2/100
2/2 [==============================] - 0s 12ms/step - loss: 0.6997 - accuracy: 0.5000
Epoch 3/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6994 - accuracy: 0.5000
Epoch 4/100
2/2 [==============================] - 0s 5ms/step - loss: 0.6990 - accuracy: 0.5000
Epoch 5/100
2/2 [==============================] - 0s 8ms/step - loss: 0.7001 - accuracy: 0.5000
Epoch 6/100
2/2 [==============================] - 0s 5ms/step - loss: 0.6988 - accuracy: 0.5000
Epoch 7/100
2/2 [==============================] - 0s 7ms/step - loss: 0.6989 - accuracy: 0.5000
Epoch 8/100
2/2 [==============================] - 0s 8ms/step - loss: 0.6970 - accuracy: 0.5000
Epoch 9/100
2/2 [==============================] - 0s 6ms/step - loss: 0.6954 - accuracy: 0.5000
Epoch 10/100
2/2 [==============================] - 0s 7ms/step - loss: 0.6936 - accuracy: 0.5000
Epoch 11/100
2/2 [==========

In [429]:
hist

# Model development

In [430]:
words = pickle.load(open('/content/words.pkl','rb'))
classes = pickle.load(open('/content/classes.pkl','rb'))


In [431]:
!pip install keras

In [432]:
from keras import models
from keras.models import load_model

In [433]:
model = load_model('/content/DS_Bot.model')

In [434]:
# cleaning the sentence
def clean_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

In [435]:
# bag of words
def bag_of_words(sentence):
  sentence_words = clean_sentence(sentence)
  bag = [0] * len(words)

  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1

  return np.array(bag)     



In [442]:
def predict_class(sentence):
  bagOfWords = bag_of_words(sentence)
  res = model.predict(np.array([bagOfWords]))[0]
  threshold = 0.25

  result = [[i,r] for i,r in enumerate(res) if r> threshold]

  result.sort(key=lambda x:x[1], reverse = True)

  return_list = []
  for r in result:
    return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
  return return_list

In [449]:
def get_res(intents_list,intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['inents']

  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result    

In [450]:
while True:
  message = input()
  ints = predict_class(message)
  res = get_res(ints,intents)
  print(res)

hai
whats up
whats your name
hello
im good
hello
you
whats up
i want to know about delays
whats up


KeyboardInterrupt: ignored